In [ ]:
!pip install ipywidgets
!pip install PyWavelets
!pip install dill
!pip install einops
!pip install neurokit2

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import os
import pandas as pd
import dill as pickle
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
from tqdm import tqdm
import timeit
from torch.nn.utils import weight_norm
import h5py
from einops.layers.torch import Rearrange, Reduce
from torch import einsum

from dl_dataloaders import *
from dl_models import *
from utils import *

In [ ]:
class options:
    def __init__(self):
        self.device ='cuda:3'
        
        self.data_dirpath = '../../datasets/vitaldb/'
        self.dataset_filename = 'vitaldb'    
        self.dataset_name = 'vitaldb'
        self.filter_name = 'zhang21'
        self.alg_name = 'jeong21'
        
        self.numWorkers = 0
        self.numEpoch = 20
        self.bs = 28
        
        self.loss_fn = loss_fn
        self.optimizer = torch.optim.Adam
        self.lr = 1e-3
        self.opt_wdecay = 2e-5
        
        self.train_pct = 0.6
        self.val_pct = 0.2
        self.test_pct = 0.2
        
        self.dataloader = jeong21_dataloader
        self.model = jeong21
        if self.model == jeong21:
            self.model_params = {}
            self.optimizer_params = {'lr':self.lr, 'weight_decay':self.opt_wdecay}
            self.schedule = identity
            self.schedule_params = {}
        elif self.model == huang22:
            self.model_params = {'in_channel':12, 'num_patches':128, 'num_classes':2,
                                   'dim':256, 'depth':6, 'attn_dim':64}
            self.optimizer_params = {'betas':(0.9, 0.98), 'eps':1e-09}
            self.schedule = identity
            self.schedule_params = {}
#             self.schedule = ScheduledOptim
#             self.schedule_params = {'d_model':1000, 'n_warmup_steps':800}
        
        self.sbp_model_names = [self.alg_name]
        self.dbp_model_names = [self.alg_name]
        
def identity(x):
    return x
        
opt = options()

In [ ]:
start = timeit.default_timer()
metadata = pd.DataFrame()  
    
fpath = opt.data_dirpath + opt.dataset_filename + '.h5'
with h5py.File(fpath, 'r') as f:
    data = f
    idxs = np.arange(len(data.keys()))
    np.random.shuffle(idxs)
    ks = np.array(list(data.keys()))
    train_idxs = ks[idxs[np.arange(0, int(opt.train_pct*len(idxs)))]]
    val_idxs = ks[idxs[np.arange(int(opt.train_pct*len(idxs)), int((opt.train_pct + opt.val_pct)*len(idxs)))]]
    test_idxs = ks[idxs[np.arange(int((opt.train_pct + opt.val_pct)*len(idxs)), len(idxs))]]

partition = {}
partition['train'] = train_idxs
partition['val'] = val_idxs
partition['test'] = test_idxs
params_train = {'batch_size': opt.bs,
              'shuffle': True,
              'num_workers': opt.numWorkers
             }
params_val = {'batch_size': 1,
              'shuffle': True,
              'num_workers': opt.numWorkers
             }
params_test = {'batch_size': 1,
              'num_workers': opt.numWorkers
             }
train_loader = torch.utils.data.DataLoader(opt.dataloader(fpath, partition['train'], opt), **params_train)
val_loader = torch.utils.data.DataLoader(opt.dataloader(fpath, partition['val'], opt), **params_val)
test_loader = torch.utils.data.DataLoader(opt.dataloader(fpath, partition['test'], opt), **params_test)

### Compute Data Distribution
# tr_sbps = np.array([])
# tr_dbps = np.array([])
val_sbps = np.array([])
val_dbps = np.array([])
te_sbps = np.array([])
te_dbps = np.array([])
# for _, y_train in train_loader:
#     tr_sbps = np.append(tr_sbps, y_train[:, 0].detach().cpu())
#     tr_dbps = np.append(tr_dbps, y_train[:, 1].detach().cpu())
for _, y_val in val_loader:
    val_sbps = np.append(val_sbps, y_val[:, 0].detach().cpu())
    val_dbps = np.append(val_dbps, y_val[:, 1].detach().cpu())
for _, y_test in test_loader:
    te_sbps = np.append(te_sbps, y_test[:, 0].detach().cpu())
    te_dbps = np.append(te_dbps, y_test[:, 1].detach().cpu())
# tr_sd_sbp = tr_sbps.std()
# tr_m_sbp = tr_sbps.mean()
# tr_sd_dbp = tr_dbps.std()
# tr_m_dbp = tr_dbps.mean()
val_sd_sbp = val_sbps.std()
val_m_sbp = val_sbps.mean()
val_sd_dbp = val_dbps.std()
val_m_dbp = val_dbps.mean()
te_sd_sbp = te_sbps.std()
te_m_sbp = te_sbps.mean()
te_sd_dbp = te_dbps.std()
te_m_dbp = te_dbps.mean()

In [ ]:
### Training
# torch.multiprocessing.set_start_method('spawn', force=True)
model = opt.model(**opt.model_params).to(opt.device)
optimizer = opt.schedule(opt.optimizer(model.parameters(), **opt.optimizer_params), **opt.schedule_params)

# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_dir = './models'
save_model_name = opt.dataset_name + '_' + opt.filter_name + '_' + opt.alg_name
writer = SummaryWriter('./runs/' + save_model_name + '_{}'.format(timestamp))
epoch_number = 0

EPOCHS = opt.numEpoch
best_vloss = 1e10
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    model.train()
    running_loss = 0.0
    last_loss = 0.0

    i = 0
    for X_train, y_train in train_loader:
        optimizer.zero_grad()
        outputs = model(X_train.float().to(opt.device))
        loss = opt.loss_fn(outputs, y_train.to(opt.device))
        loss.backward()
        optimizer.step()
        i+=1
        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_number * len(train_loader) + i + 1
            writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.0
    avg_loss = last_loss

    model.eval()    
    vlosses_s = np.array([])
    vlosses_d = np.array([])
    running_vloss = 0.0
    for X_val, y_val in val_loader:
        voutputs = model(X_val.float().to(opt.device))
        vloss = opt.loss_fn(voutputs, y_val)
        vloss_s, vloss_d = me_loss(voutputs, y_val)
        vlosses_s = np.append(vlosses_s, vloss_s.detach().cpu().numpy())
        vlosses_d = np.append(vlosses_d, vloss_d.detach().cpu().numpy())
        running_vloss += vloss.item()
    EDd = val_sd_dbp/vlosses_d.std()
    EDs = val_sd_sbp/vlosses_s.std()
    avg_vloss = running_vloss/len(val_loader)/params_val['batch_size']
    print('train_loss = {} \n valid_loss = {} \n EDs = {} \n EDd = {}'.format(avg_loss, avg_vloss, EDs, EDd))

    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = model_dir + os.sep + opt.dataset_name + '_' + opt.filter_name + '_' + opt.alg_name
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [ ]:
### Testing

opt.device= 'cpu'
# model = opt.model(**opt.model_params).to(opt.device)
model_dir = './models'
model_path = model_dir + os.sep + opt.dataset_name + '_' + opt.filter_name + '_' + opt.alg_name
saved_model = opt.model(**opt.model_params).to(opt.device)
saved_model.load_state_dict(torch.load(model_path))
saved_model.eval() 

running_te_loss = 0.0
running_s_loss = 0.0
running_d_loss = 0.0
te_losses_s = np.array([])
te_losses_d = np.array([])
sbp_ests = np.array([])
dbp_ests = np.array([])
sbp_gts = np.array([])
dbp_gts = np.array([])
for i in range(0, 10):
    i=0
    for X_test, y_test in test_loader:
        te_outputs = saved_model(X_test.float().to(opt.device))
        te_loss = opt.loss_fn(te_outputs, y_test)
        te_loss_s, te_loss_d = me_loss(te_outputs, y_test)
        te_losses_s = np.append(te_losses_s, te_loss_s.detach().cpu().numpy())
        te_losses_d = np.append(te_losses_d, te_loss_d.detach().cpu().numpy())
        sbp_ests = np.append(sbp_ests, te_outputs[0, 0].detach().cpu().numpy())
        dbp_ests = np.append(dbp_ests, te_outputs[0, 1].detach().cpu().numpy())
        sbp_gts = np.append(sbp_gts, y_test[0, 0].detach().cpu().numpy())
        dbp_gts = np.append(dbp_gts, y_test[0, 1].detach().cpu().numpy())
        running_te_loss += te_loss
        i+=1
te_EDd = te_sd_dbp/te_losses_d.std()
te_EDs = te_sd_sbp/te_losses_s.std()
avg_te_loss = running_te_loss.detach().cpu().item() / (i + 1)
avg_s_loss = abs(te_losses_s).mean()
avg_d_loss = abs(te_losses_d).mean()
print('test_loss = {} \n sbp_loss = {} \n dbp_loss = {} \n EDs = {} \n EDd = {}'.format(avg_te_loss, avg_s_loss, avg_d_loss, te_EDs, te_EDd))     

sbp_model_results = {}
dbp_model_results = {}
sbp_std = te_sd_sbp
dbp_std = te_sd_dbp
sbp_model_result = {}
sbp_model_result['raw ests'] = sbp_ests
sbp_model_result['raw gts'] = sbp_gts
sbp_model_result['bias'] = (sbp_ests - sbp_gts).mean()
sbp_model_result['err std'] = (sbp_ests - sbp_gts).std()
sbp_model_result['ED'] = sbp_std/(sbp_ests - sbp_gts).std()
sbp_model_result['dist std'] = sbp_std
sbp_model_results[opt.alg_name] = sbp_model_result
dbp_model_result = {}
dbp_model_result['raw ests'] = dbp_ests
dbp_model_result['raw gts'] = dbp_gts
dbp_model_result['bias'] = (dbp_ests - dbp_gts).mean()
dbp_model_result['err std'] = (dbp_ests - dbp_gts).std()
dbp_model_result['ED'] = dbp_std/(dbp_ests - dbp_gts).std()
sbp_model_result['dist std'] = sbp_std
dbp_model_results[opt.alg_name] = dbp_model_result

stop = timeit.default_timer()
print('Time: ', stop - start)

In [ ]:
pickle_dict = {'sbp': sbp_model_results, 'dbp': dbp_model_results, 'opt': opt}
with open('../results/training/' + opt.dataset_name + '_' + opt.filter_name + '_' + opt.alg_name + '.pickle', 'wb') as f:
    pickle.dump(pickle_dict, f, protocol=pickle.HIGHEST_PROTOCOL)